$$
\\ \epsilon_t=P_{i~U^{(t)}}[f_t(x_i)\neq y_i]
\\ \text{train }f_2x\text{ on new training set that fails }f_1x\text{. found with }\epsilon
\\ \epsilon_1 = \frac {\sum_n u_n^{(1)} \delta (f_1(x_n)!=y_n)} {Z_1}
\\\text{where } Z_1 = \sum_{n=1}^N u_n^{(1)},\epsilon \lt 0.5
$$

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#data preprocessing like in 3
df = pd.read_csv("train.csv")
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
df = df.dropna()
df = df.replace('female', 1).replace('male', 0).replace('C', 1).replace('Q', 2).replace('S', 3)
#split the training and testing data
train, test = train_test_split(df, test_size=0.2, random_state=0)
#initialize the decision tree
dtc = DecisionTreeClassifier(max_depth=2, random_state=1, criterion='entropy')
#set data and target equal to my input and expected output
data = train.drop(columns=['Survived']).to_numpy()
target = train['Survived'].to_numpy()
#alter the expected output so it classifies the negative class as -1 instead of 0
for t in range(len(target)):
    if target[t] == 0:
        target[t] = -1
data2 = test.drop(columns=['Survived']).to_numpy()
target2 = test['Survived'].to_numpy()
#alter the expected output so it classifies the negative class as -1 instead of 0
for t in range(len(target2)):
    if target2[t] == 0:
        target2[t] = -1
#fits the tree to it's own training set
dtc.fit(data, target)
#prints the original accuracy
print(dtc.score(data,target))
print(dtc.score(data2,target2))

In [ ]:
#this function calculates the current accuracy of the sum of the weighted models
#Hi is the sum of the current votes for each sample(a list of len s)
#tar is the expected target values
#s is the length of the dataset
def currAcc(Hi,tar,s):
    finalAcc = 0
    for j in range(s):
        #because the sample classifications wont always be exactly -1/1
        #I improvised this to treat the sign as the label
        if (Hi[j] > 0 and tar[j] > 0) or (Hi[j] <= 0 and tar[j] <= 0):
            finalAcc += 1
    return finalAcc/s

In [ ]:
#set N as the number of samples in train M for test
N = len(train)
M = len(test)
#set u as the list of weight distributions for training
u = np.zeros((1,N))
for i in range(N):
    u[0][i] = 1/N
Z = 1.0
#list of our lagrange multipliers and error rates
a = []
e = []
alpha = 0.0
epsilon = 0.0
#variables for the final hypothesis H is where the votes are stored for each sample
#and temp is each weighted models prediction for a sample
temp = 0
#initialize H and Htest
H = [0]*N
H2 = [0]*M
#list of the test and train errors
trainErrs = []
testErrs = []
#iterate for 500 rounds
for R in range(1,501):
    #set Z as the normalizer
    Z = sum(u[0])
    #here I initialize epsilon which will be added to the list of error rates
    epsilon = 0.0
    for sample in range(N):
        epsilon += u[0][sample]*target[sample]*dtc.predict(data[sample].reshape(1, -1))
    epsilon = 0.5 - epsilon/2
    #here I initialize aplha which will be added to the list of error rates
    alpha = 0.5*np.log((1-epsilon)/epsilon)
    a.append(alpha)
    e.append(epsilon)
    #here I update the sample weights using the lagrange multiplier above as well as the latest weighted model
    for sample in [range(N)]:
        u[0][sample] *= np.exp(-alpha*target[sample]*dtc.predict(data[sample]))/Z
    #here the model is refitted and the votes for the model are cast or added to the final hypothesis
    dtc.fit(data, target, sample_weight=u[0])
    #for train
    for i in range(N):
        temp = dtc.predict(data[i].reshape(1, -1))
        H[i] += alpha*temp
    #for test
    for i in range(M):
        H2[i] += alpha*dtc.predict(data2[i].reshape(1, -1))
    #here I append the errors at this point to the lists using my helper function currAcc
    trainErrs.append(1-currAcc(H,target,N))
    testErrs.append(1-currAcc(H2,target2,M))
    #dtc.fit(data, target)

In [ ]:
plt.ylabel('error')
plt.xlabel('rounds')
plt.plot(trainErrs, label='training set')
plt.plot(testErrs, label='testing set')
plt.legend()
plt.show()
print('final train acc %f'%(1-trainErrs[-1]))
print('final test acc %f'%(1-testErrs[-1]))
print('best train acc %f'%(1-min(trainErrs)))
print('best test acc %f'%(1-min(testErrs)))